# Нейросетевые модели поиска. Часть II. Переранжирование.
## Домашнее задание

В этом задании вам предстоит обучить модели переранжирования с архитектурой cross-encoder и bi-encoder на датасете [VK MARCO](https://cloud.mail.ru/public/MQ3H/GVGeAWZoj). За его решение можно получить до 10 баллов, а также еще 10 бонусных за выполнение дополнительных заданий.

**Вам надо:**

- Форкнуть эту репу;
- Создать бранч, в котором вы дальше будете работать;
- Выполнить все или часть заданий ноутбука;
- Запушить ваш бранч и поставить Pull Request.

Проверяющий счекаутит вашу бранчу и проверит работу.

**Замечания:**

- При выполнении задания можете использовать код из соответствующего семинара;
- Вам предстоит сделать как минимум 2 обучения (а при выполнении дополнительных заданий и того больше), постарайтесь переиспользовать код и минимизировать копипасты;
- Старайтесь писать чистый и понятный код, не оставляйте в ноутбуке лишний мусор (выводы ячеек с pip install, ненужный закомментированный код и т.д.);
- При сдаче дз сохраните выходы ячеек с логами обучения и графиками. Если для построения графиков вы использовали tensorboard, то вставьте графики в ноутбук в виде скриншотов;
- Итоговый ноутбук должен быть работоспособен, то есть проверяющий должен иметь возможность запустить все его ячейки в том порядке, в котором они распологаются в ноутбуке, и получить результаты, аналогичные вашим.


### Данные [1 балл]

Скачайте и распакуйте датасет [VK MARCO](https://cloud.mail.ru/public/MQ3H/GVGeAWZoj). Архив содержит следующие файлы (формат аналогичен MS MARCO):
* vkmarco-docs.tsv - tsv с текстами документов;
* vkmarco-doctrain-queries.tsv - tsv с текстами запросов;
* vkmarco-doctrain-qrels.tsv - tsv с оценками релевантности запроса и документа;
* Аналогичный набор файлов для валидации.

Загрузите данные трейна и валидации в pandas.DataFrame так, чтобы он содержал следующие колонки:
* qid - id запроса;
* query - текст запроса;
* text - текст документа;
* label - оценка релевантности запроса и документа.

In [ ]:
import pandas as pd
import numpy as np

**Download file from cloud.mail.ru from linux console with bash script:**

https://gist.github.com/cronfy/00e23e126d4fbf3fab37392e414f0e4c

In [ ]:
!wget !wget https://raw.githubusercontent.com/MaxTimoshkin/vk-msu-ir-course-spring-2024/TimoshkinMaksim/homeworks/nn-reranking/dl-cloud-mail-ru.sh

!chmod +x dl-cloud-mail-ru.sh

In [ ]:
#Downloading file from cloud.mail.ru from linux console with bash script:
!./dl-cloud-mail-ru.sh https://cloud.mail.ru/public/MQ3H/GVGeAWZoj vkmarcoranking-v1-document-noeval.tar

In [ ]:
!tar -xvf vkmarcoranking-v1-document-noeval.tar
!rm vkmarcoranking-v1-document-noeval.tar

In [ ]:
!gunzip vkmarcoranking-v1-document/vkmarco-docs.tsv.gz
!gunzip vkmarcoranking-v1-document/vkmarco-doctrain-queries.tsv.gz
!gunzip vkmarcoranking-v1-document/vkmarco-doctrain-qrels.tsv.gz

!gunzip vkmarcoranking-v1-document/vkmarco-docs-lookup.tsv.gz
!gunzip vkmarcoranking-v1-document/vkmarco-docdev-queries.tsv.gz
!gunzip vkmarcoranking-v1-document/vkmarco-docdev-qrels.tsv.gz

In [ ]:
!ls vkmarcoranking-v1-document

vkmarco-docdev-qrels.tsv    vkmarco-docs.tsv
vkmarco-docdev-queries.tsv  vkmarco-doctrain-qrels.tsv
vkmarco-docs-lookup.tsv     vkmarco-doctrain-queries.tsv


In [ ]:
PATH = 'vkmarcoranking-v1-document'

train_docs_file = f'{PATH}/vkmarco-docs.tsv'
train_queries_file = f'{PATH}/vkmarco-doctrain-queries.tsv'
train_qrels_file = f'{PATH}/vkmarco-doctrain-qrels.tsv'

val_docs_file = f'{PATH}/vkmarco-docs-lookup.tsv'
val_queries_file = f'{PATH}/vkmarco-docdev-queries.tsv'
val_qrels_file = f'{PATH}/vkmarco-docdev-qrels.tsv'

In [ ]:
df_docs = pd.read_csv(train_docs_file, sep='\t', header=None)
train_df_queries = pd.read_csv(train_queries_file, sep='\t', header=None)
train_df_qrels = pd.read_csv(train_qrels_file, sep=' ', header=None)

In [ ]:
val_df_docs = pd.read_csv(val_docs_file, sep='\t', header=None)
val_df_queries = pd.read_csv(val_queries_file, sep='\t', header=None)
val_df_qrels = pd.read_csv(val_qrels_file, sep=' ', header=None)

In [ ]:
def join_df(df_docs, df_queries, df_qrels):
    df_docs.rename(columns={0 : 'doc_id', 1 : 'url', 2 : 'title', 3 : 'text'}, inplace=True)
    if 'url' in df_docs.columns:
        df_docs.drop('url', axis=1, inplace=True)

    df_queries.rename(columns={0: "qid", 1: "query"}, inplace=True)

    df_qrels.rename(columns={0:'qid', 1:'_', 2:'doc_id', 3:'label'}, inplace=True)
    df_qrels.drop(['_'], axis=1, inplace=True)

    df = df_qrels.join(df_queries.set_index(['qid']), on='qid')
    df = df.join(df_docs.set_index('doc_id'), on='doc_id')
    return df[['qid', 'query', 'text', 'label']]

In [ ]:
train_df = join_df(df_docs, train_df_queries, train_df_qrels)
valid_df = join_df(df_docs, val_df_queries, val_df_qrels)

In [ ]:
train_df

,qid,query,text,label
0,1,0 00 дом muzono net raim feat artur adil скачать,Скачивай и слушай 🎧 raim дом o2 2018 на Zvuk.t...,1
1,1,0 00 дом muzono net raim feat artur adil скачать,Скачать песню RaiM feat. Artur & Adil - Дом бе...,3
2,1,0 00 дом muzono net raim feat artur adil скачать,Скачать песню RaiM ft. Artur & Adil - Дом в Фо...,2
3,1,0 00 дом muzono net raim feat artur adil скачать,Скачать RaiM feat. Artur & Adil - Дом в mp3 фо...,2
4,1,0 00 дом muzono net raim feat artur adil скачать,Здесь можете бесплатно скачать Дом - RaiM feat...,2
...,...,...,...,...
1002625,42770,հեպ 9 դասագիրք,null Մայիսի 9 << Մայիս >> Կի Եկ Եք Չո Հի Ու Շա...,1
1002626,42770,հեպ 9 դասագիրք,null Հայերեն English 02 Сентября 2022 | Пятниц...,0
1002627,42771,美女的胸,系统漫画《大英雄的女友超级凶》，简介：根据阿里巴巴文学二次元签约作者大仙的同名小说改编。他是...,0
1002628,42771,美女的胸,Смотрите онлайн 善良的女祕書 [中文字幕].. 20 мин 14 с. В...,0


In [ ]:
valid_df

,qid,query,text,label
0,5,05 05 2019 выходной,"Выходные, праздники, не рабочие дни в 2019 год...",2
1,5,05 05 2019 выходной,Официальные праздники и выходные в мае 2019 го...,2
2,5,05 05 2019 выходной,null Как отдыхают в мае 2019 года в России и с...,2
3,5,05 05 2019 выходной,"О рабочем времени в 2022 году 📅: выходные, пра...",1
4,5,05 05 2019 выходной,В мае 2019 года у украинцев будут дополнительн...,1
...,...,...,...,...
153367,42757,югра безопасность урай реквизиты,"Информация о компании ООО ""ЮГРА-БЕЗОПАСНОСТЬ"" ...",3
153368,42757,югра безопасность урай реквизиты,"ООО ""ЮГРА-БЕЗОПАСНОСТЬ"" - г. Урай - ОГРН 10486...",3
153369,42757,югра безопасность урай реквизиты,"ООО ""ЮГРА-БЕЗОПАСНОСТЬ"", г. Урай: реквизиты, р...",2
153370,42757,югра безопасность урай реквизиты,"ИНН, ОГРН, юридический адрес, ФИО директора ко...",2


In [ ]:
!rm -r vkmarcoranking-v1-document

In [ ]:
# Сохранение файлов для последующего создания датасетов (т.к. иначе не хватает оперативной памяти)
train_df.to_csv('train_df.csv', sep='\t', index=False)
valid_df.to_csv('valid_df.csv', sep='\t', index=False)

In [ ]:
!mkdir /kaggle/working/train_df

!wget https://raw.githubusercontent.com/MaxTimoshkin/vk-msu-ir-course-spring-2024/TimoshkinMaksim/homeworks/nn-reranking/dataset-metadata.json

!mv dataset-metadata.json train_df/dataset-metadata.json

In [ ]:
!mv /kaggle/working/train_df.csv  /kaggle/working/train_df/train_df.csv
!mv /kaggle/working/valid_df.csv  /kaggle/working/train_df/valid_df.csv

In [ ]:
!kaggle datasets create -p /kaggle/working/train_df

Starting upload for file valid_df.csv
100%|██████████████████████████████████████| 1.91G/1.91G [00:43<00:00, 47.3MB/s]
Upload successful: valid_df.csv (2GB)
Starting upload for file train_df.csv
100%|██████████████████████████████████████| 12.4G/12.4G [04:35<00:00, 48.4MB/s]
Upload successful: train_df.csv (12GB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/datasets/maxtimoshkin/df-train


In [ ]:
# Звгрузка данных из датасета
train_df = pd.read_csv('/kaggle/input/df-train/train_df.csv', sep='\t')
valid_df = pd.read_csv('/kaggle/input/df-train/valid_df.csv', sep='\t')

### Baselines \[2 балла\]

Реализуйте подсчет метрик MRR@k и NDCG@k. Посчитайте их значения на валидационном сете для нескольких бейзлайнов:
1. Модель, делающая случайное предсказание;
1. BM25.

In [ ]:
import torch
from torchmetrics.retrieval import RetrievalNormalizedDCG
from torchmetrics.retrieval import RetrievalMRR

In [ ]:
def MRR(preds, targets, qids):
    return RetrievalMRR(top_k=10, empty_target_action='neg')(
        torch.Tensor(preds),
        torch.LongTensor(targets >= 2),
        indexes=torch.LongTensor(qids - min(qids))
    )

def NDCG(preds, targets, qids):
    return RetrievalNormalizedDCG(top_k=10, empty_target_action='skip')(
        torch.Tensor(preds),
        torch.LongTensor(targets.replace(1, 0)),
        indexes=torch.LongTensor(qids - min(qids))
    )

In [ ]:
# Случайное предсказание
np.random.seed(42)

random_preds = np.random.random(valid_df.shape[0])

mrr  = MRR(random_preds, valid_df['label'], valid_df['qid'])
ndcg = NDCG(random_preds, valid_df['label'], valid_df['qid'])

print(f'MRR = {mrr}')
print(f'NDCG = {ndcg}')

MRR = 0.5713226199150085
NDCG = 0.4331240952014923


In [ ]:
!pip install rank-bm25

In [ ]:
from rank_bm25 import BM25Okapi

corpus = valid_df['text'].values
tokenized_corpus = [doc.split(" ") for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
def get_bm25_scores(test_data):
    queries = test_data['query'].unique()
    bm25_preds = np.zeros(len(test_data))
    for q in tqdm.tqdm(queries):
        tokenized_query = q.split(" ")
        doc_scores = bm25.get_scores(tokenized_query)
        mask = test_data['query'] == q
        bm25_preds[mask] = doc_scores[mask]
    return bm25_preds

In [ ]:
import tqdm

bm25_preds = get_bm25_scores(valid_df)

100%|██████████| 5000/5000 [41:48<00:00,  1.99it/s]  


In [ ]:
# BM25 предсказание
mrr  = MRR(bm25_preds, valid_df['label'], valid_df['qid'])
ndcg = NDCG(bm25_preds, valid_df['label'], valid_df['qid'])

print(f'MRR = {mrr}')
print(f'NDCG = {ndcg}')

MRR = 0.5798457264900208
NDCG = 0.45509999990463257


### Обучение cross-encoder \[2 балла\]

Выполните следующие задания:
* Обучите модель xlm-roberta-base в фомате cross-encoder.
* Постройте графики зависимости метрик от количества пройденных шагов.
* Посчитайте метрики для финальной модели.
* Сколько времени заняло обучение?

### Обучение bi-encoder \[4 балла\]

Вспомним, что разница между cross-encoder и bi-encoder моделями заключается способе получения скора релевантности:

| Cross-encoder | Bi-encoder |

Выполните следующие задания:
* Внесите необходимые изменения в код и обучите xlm-roberta-base в фомате bi-encoder.
* Постройте графики зависимости метрик от количества пройденных шагов.
* Посчитайте метрики для финальной модели.
* Сколько времени заняло обучение?

In [ ]:
# YOUR CODE HERE

### Сравнение cross-encoder и bi-encoder архитектур \[2 балла\]

Выполните следующие задания:
* Сравните графики обучения моделей, скорость обучения и скорость инфернса.
* Напишите вывод.

### Дополнительно \[10 баллов\]

Вы можете получить дополнительные баллы, выполнив одно или несколько заданий:
* Обучите cross-encoder в fp16 с использованием AMP, посчитайте метрики, есть ли изменения? **[1 балла]**
* Реализуйте семплирование примеров с учетом их длинны, насколько ускорилось обучение cross-encoder? **[1 балла]**
* Попробуйте увеличить метрики bi-encoder за счет использования другого претрейна. Кроме xlm-roberta-base на [Huggingface](https://huggingface.co/models) есть большое количество предобученных моделей BERT. Датасет содержит тексты на русском языке, поэтому стоит обратить внимание на русскоязычные или мультиязычные претрейны. Если вам удасться улучшить метрику, то предположите, какие отличия претрейна на это повлияли. **[2 балла]**
* Попробуйте разморозить больше параметров модели (например, последний трансформерный слой). Как изменилось качество и скорость обучения? **[2 балла]**
* Попробуйте использовать listwise лосс для обучения модели, как изменились метрики? **[4 балла]**

In [ ]:
# YOUR CODE HERE